In [2]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from lime.lime_text import LimeTextExplainer

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [1]:
from transformers import pipeline

/home/xinyu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-19 16:07:43.304656: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-19 16:07:44.086848: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-19 16:07:44.086864: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-11-19 16:07:

KeyboardInterrupt: 

In [11]:
tokenizer.pad_token = tokenizer.eos_token

In [11]:
import numpy as np

In [ ]:
text_generator = pipeline("text-generation", model="gpt2")

In [ ]:
def generate_text(text):
    generated_texts = text_generator(text, max_length=50, num_return_sequences=1)
    return generated_texts[0]['generated_text']

In [17]:
def predict_proba(texts):
    outputs = []
    for text in texts:
        inputs = tokenizer.encode_plus(text, return_tensors='pt', truncation=True, padding='max_length', max_length=tokenizer.model_max_length)
        with torch.no_grad():
            logits = model(**inputs).logits
        last_logits = logits[:, -1, :]
        probabilities = torch.nn.functional.softmax(last_logits, dim=-1)
        outputs.append(probabilities.cpu().numpy()[0])
    return np.array(outputs)

In [6]:
text_to_explain = "How you doing today"

In [7]:
explainer = LimeTextExplainer(class_names=['token_probability'])

In [19]:
exp = explainer.explain_instance(text_to_explain, predict_proba, num_features=6,num_samples=50)

In [20]:
print(exp.as_list())

[('How', -0.003474972971109041), ('doing', 0.0015593182323584952), ('today', 0.0014845432256675422), ('you', 0.0013364085007617492)]


In [2]:
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
from sklearn.linear_model import LogisticRegression
from lime.lime_text import LimeTextExplainer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2ForSequenceClassification.from_pretrained('gpt2')
model.eval()


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)

In [9]:
def predict_proba(texts):
    outputs = []
    for text in texts:
        inputs = tokenizer.encode_plus(text, return_tensors='pt', truncation=True, padding='max_length', max_length=tokenizer.model_max_length)
        with torch.no_grad():
            logits = model(**inputs).logits
        probabilities = torch.nn.functional.softmax(logits, dim=1).cpu().numpy()
        outputs.append(probabilities.squeeze(0))
    return np.array(outputs)

In [5]:
tokenizer.pad_token = tokenizer.eos_token

In [12]:
explainer = LimeTextExplainer(class_names=["A", "B", "C", "D"])

text_to_explain = "What's the sum of 1 and 2? A.3 B.4 C.5 D.6"

exp = explainer.explain_instance(text_to_explain, predict_proba, num_features=6, num_samples=50)

print(exp.as_list())

[('A', -0.023329125995983697), ('sum', -0.015917483664884158), ('4', -0.01561283518289362), ('1', -0.012815235625985692), ('s', -0.011999361880431913), ('6', -0.009044679895870522)]
